# Projeto 3 de Ciência dos Dados

## Integrantes

- Bruno Domingues
- Stefano Moretti
- Thomas Queiroz

### Introdução

O projeto consiste na montagem de um classificador que tem como input uma foto de uma pessoa qualquer, e com base na foto retornar o sexo biológico da pessoa na foto.

Para tal, usamos um dataset composto de 13000 fotos, chamado de "Labeled Faces in the Wild". Nele, há 13000 fotos de 5749 pessoas diferentes, e dentre estas, 1680 pessoas possuem mais de uma foto. A única "restrição" presente é que as fotos foram obtidas através do detector de faces Viola-Jones. Há quatro versões do dataset que podem ser encontradas:
- versão original
- versão com alinhamento por deep funneling
- versão com alinhamento por LFW-a
- versão com alinhamento por funneling

No caso, optamos pela última, uma vez que acreditamos que o alinhamento das faces torna o processo de detectar padrões mais fácil.

Com o dataset em mãos, começamos a adaptá-lo para nosso projeto.

### Limpeza dos dados

Primeiro, obtivemos o local onde as fotos se encontravam no computador. Em seguida, unimos as fotos com 2 arquivos txt em uma pasta, sendo que estes arquivos continham os nomes de homens e mulheres presentes no dataset, gerando nossos labels. Após isto, começamos a editar as fotos: convertemos para greyscale (pois assim ele não precisa analisar o espectro RGB), reduzimos o tamanho da foto de 250x250 para 40x40 pixels, e por fim transformamos as fotos em matrizes em que cada pixel é um elemento da matriz, de forma que o classificador possa analisar cada pixel individualmente.

In [8]:
import cv2, os
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [9]:
path_ori = os.getcwd()
# path_dataset = "C:\\Users\\thoma\\Documents\\INSPER\\2 Sem\\Ciência dos Dados\\lfw_funneled"
path_dataset = "C:\\Users\\Bruno\\Documents\\Insper\\2º Semestre\\Ciência dos Dados\\Projeto_3\\lfw_funneled"
# path_dataset = "C:\\Users\\Stefano Moretti\\Documents\\INSPER\\2 Sem\\Ciência dos Dados\\lfw_funneled"

In [10]:
initialloadTime = time()

os.chdir(path_dataset)

with open('male_names.txt','r') as m:
    male = m.readlines()
    male = [i.replace("\n","") for i in male]

with open('female_names.txt','r') as fm:
    female = fm.readlines()
    female = [i.replace("\n","") for i in female]


m = []
fm = []

for pasta in os.listdir(path_dataset):
    if "." not in pasta:
        path_folder = ".\\" + pasta
        for im in os.listdir(path_folder):
            tempIm = cv2.imread(path_folder + "\\" + im)

            tempIm = cv2.pyrDown(tempIm)
            tempIm = tempIm[20:100, 20:100]
            tempIm = cv2.pyrDown(tempIm)
            tempIm = cv2.cvtColor(tempIm, cv2.COLOR_BGR2GRAY)
            
            tempIm = tempIm.flatten()
            
            if im in male:
                m.append(tempIm)
            elif im in female:
                fm.append(tempIm)

os.chdir(path_ori)

finalloadTime = time()
f'Tempo para carregar as fotos: {finalloadTime - initialloadTime :.2f} segundos'

'Tempo para carregar as fotos: 20.72 segundos'

In [11]:
def plotImage(crop_img):
    newIm = []
    for i in range(crop_img.shape[0]):
        newIm.append([])
        for j in range(crop_img.shape[1]):
            newIm[-1].append([crop_img[i,j]]*3)
    plt.imshow(newIm)
    plt.show()

def plotImage1D(arr):
    newIm = []
    i=0
    
    while i < len(arr):
        newIm.append(arr[i:i+40])
        i+=40
    plotImage(np.array(newIm))

Após a limpeza dos dados, juntamos as fotos de homens e mulheres, e embaralhamos as fotos. Em seguida, separamos o dataset em 2 partes: 70% das fotos seriam usadas para o treino do classificador e o restante seria usado para teste.

In [12]:
X = m + fm
y = [0]*len(m) + [1]*len(fm)

c = list(zip(X, y))

np.random.shuffle(c)

X, y = zip(*c)

In [13]:
index = round(len(X) * 0.7)
X_train, X_test = X[:index], X[index:]
y_train, y_test = y[:index], y[index:]

Com o dataset de treino e teste separado, pudemos seguir para a escolha de nosso classificador.

### Classificador

Após testar diversos classificadores que podem ser encontrados na biblioteca do scikit learn, optamos por usar o Gradient Boosting Classifier, uma vez que ele nos retornou a melhor acurácia.

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

comeco_treino = time()
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)

fim_treino = time()
print(f'Tempo para treinar o modelo: {fim_treino - comeco_treino:.2f} segundos')

comeco_prev = time()
y_pred = clf.predict(X_test)

fim_prev = time()
print(f'Tempo para testar o modelo: {fim_prev - comeco_prev :.2f} segundos')


'Acurácia: {0:.2f} %'.format(100*accuracy_score(y_test, y_pred))

Tempo para treinar o modelo: 114.01 segundos
Tempo para testar o modelo: 0.05 segundos


'Acurácia: 86.80 %'

Mas afinal, o que seria o Gradient Boosting?

Antes, devemos entender o que é Gradient Descent. Explicando de forma simples, é um método de obter o mínimo de uma função. Para tal, usamos a seguinte fórmula:

$ x^{(i)} = x^{(i−1)} − η*\frac{df(x^{(i−1)})}{dx} $

onde $x^{(i)}$ é o próximo valor a ser obtido na função, $x^{(i-1)}$ é o valor inicial escolhido e η é uma constante positiva que multiplica a derivada da função que estamos analisando no ponto $x^{(i-1)}$. Este processo é repetido n vezes até que $x^{(i)} = x^{(i−1)}$. Isto também pode ser feito com um vetor x, e no caso ao invés de usarmos a derivada usamos o gradiente deste vetor (por isto o nome gradient), de modo que movimentando o vetor x, cada componente do vetor vise minimizar a função, gerando a fórmula a seguir:

$ x^{(i)}_j = x^{(i−1)}_j − η*\frac{\partial f(x^{(i−1)})}{\partial x_j} $

Com isto, podemos seguir para a ideia de Gradient Boosting. Imagine que nossa função é uma função que é inversamente proporcional a acurácia de nosso classificador: quanto pior ele for, maior será o valor da função. Esta função é chamada de função loss em machine learning. Para podermos usar este método, a função loss **deve ser** diferenciável. Um exemplo seria o quadrado do erro (diferença entre o valor real e o valor previsto).

$ L = (y_i - h(x_i))^2 $

Logo, queremos minimizar esta função loss, dada por $\sum_{i=1}^{N} L(y_i,h(x_i))$ (ou seja, a soma de todas as "perdas" ao longo da função.), onde $y_i$ é o valor esperado, $h(x_i)$ é o valor previsto, $h(x)$ é o classificador e N é o número total de dados encontrados. Se queremos que o somatório seja mínimo, queremos portanto que a função do classificador seja mínima, de modo que gere uma função loss mínima.

Voltando então ao Gradient Descent, seguimos os seguintes passos:

1. Comece com $h^{(0)}(x)=c$, uma constante, de modo que c minimize a função $f(x)$, por exemplo, substituir no lugar de $h(x_i)$ na função loss.
2. Na i-ésima iteração, para j=1,2,...,N computar $r_{ji} = \frac{\partial{−L(y_j, h^{(i−1)}(x_j))}}{\partial{h^{(i−1)}(x_j)}}$. Isto é possível pois assumimos que L é uma função diferenciável - no exemplo do erro quadrado, $\frac{\partial L}{\partial h} = −2(y−h)$, pois estamos somente substituindo os valores $y_j$ and $h^{(i−1)}(x_j)$ na derivada. Isto é análogo ao processo que fizemos com os componentes do vetor anteriormente.
3. O passo anterior nos retorna um valor $r_{ji}$ para cada ponto $j$. Logo, temos um set de tuples $(x_j,r_{ji})$. Usamos este set de pontos como dados de treinamento que constroem uma árvore de regressão que consegue prever $r_{ji}$ dado $x_j$. Esta árvore se aproxima ao gradiente. Imagine esta árvore como uma expressão "caixa preta" que retorna o $r_{ji}$ dado $x_j$. Ela toma o lugar da expressão $\frac{\partial f(x^{(i−1)})}{\partial x_j}$ que vimos no Descent, de modo que ela "encorpore" o gradiente para todos os pontos $x_j$. Iremos nos referir a esta árvore por $T^{(i)}_g$ (g de gradiente, i é a iteração). Como antes, queremos que esta árvore-gradiente tenha um papel importante na equação, mas ainda falta encontrar o η.
4. Assuma que a árvore $T^{(i)}_g$ tenha K folhas. Sabemos que as folhas de uma árvore fragmentam o espaço em regiões disjuntas. Iremos nos referir a estas regiões por $R_k$, para k=1,2,...,K. Se "soltarmos" cada ponto $x_j$ do topo da árvore $T^{(i)}_g$, o ponto irá cair em alguma região $R_k$. Queremos agora associar uma constante $η_k$ para cada região $R_k$ tal que a perda em uma região, definida como: $\sum_{x_j \in R_k} L(y_j,h^{(i−1)}(x_j)+η_k)$ seja mínima. Estes são solucinados como problemas de minimização independentes entre si para as k regiões. Note que agora temos uma árvore fornecendo regiões $R_k$ bem definidas, e temos os valores das constantes $η_k$ associado a cada região. Esta combinação pode ser vista como outra árvore: o tronco é dado por $T^{(i)}_g$ e as folhas são dadas por $η_k$ servindo como previsões.
5. Por fim, chegamos ao passo de atualização: $h^{(i)}(x) = h^{(i−1)}(x) + \sum_{k} η_{k}I(x \in R_k)$. Aqui, $I(x \in R_k)$ serve como uma função indicadora que retorna 1 caso $x$ caia na região $R_k$, 0 caso contrário. (Isto na verdade é uma forma elegante de dizer que para um ponto $x$ o segundo termo é $η_k$ que corresponde a região em que o ponto caiu. Este segundo termo é efetivamente uma árvore que deriva de $T^{(i)}_g$. Agora deve ficar evidente o porque de $n_k$ ter sido determinado da forma que foi: a minimização do passo anterior e a função de atualização tem a mesma forma, logo, a função de atualização já tem a menor perda possível. Note a similaridade desta equação ao Descent: você tem o gradiente e o η. Também note que na verdade não foi adicionado nada neste passo de atualização: o que estamos dizendo é que se deseja computar o $h^{(i)}(x)$, compute $h^{(i-1)}(x)$, e adicione o $η_k$ que você obtiver ao passar x ao longo da árvore representada pelo segundo termo).
6. Repetir o passo 2 quantas vezes você quiser iterar.
7. Finalmente retornar $h^{(M)}(x)$ como seu preditor. Como em cada iteração sua única atualização à função é adicionar uma árvore no passo 5, o que você retornar vai ser a soma das árvores prévias ou um monte de árvores que somadas formam $h^{(M)}(x)$.

A diferença crucial entre o Gradient Boosting e o Descent é que ao entregar um ponto, na Descent você recebe um $h^{(i)}$ e no Boosting você recebe i árvores e uma constante c.

Podemos assumir que esse modelo é o mais adequado para nosso projeto pois ao realizar a análise de cada pixel, o processo de gerar n árvores de decisão é capaz de retornar melhor onde cada pixel se encaixaria, e portanto, dizer melhor se a foto se assemelha mais a um homem ou uma mulher.

Link para o dataset: http://vis-www.cs.umass.edu/lfw/